In [1]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [70]:
from openai import OpenAI

In [82]:
client=OpenAI() #I removed the secret key, otherwise github wouldn't let me push to changes

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [2]:
from elasticsearch import Elasticsearch
es_client= Elasticsearch('http://localhost:9200')
es_client.info()

ObjectApiResponse({'name': 'c6c163de3cc2', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'BPYr-E-8Ry2BT9wDhfIpcA', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [4]:
index_settings={
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name="course"

In [5]:
from tqdm.auto import tqdm

/usr/local/python/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
for doc in tqdm(documents):
    es_client.index(index=index_name,document=doc)

100%|█████████████████████| 948/948 [00:20<00:00, 46.36it/s]


In [7]:
query="How do I execute a command in a running docker container?"

In [8]:
search_query={
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },

        }
    }
}

In [9]:
response= es_client.search(index=index_name, body=search_query)

In [10]:
response

ObjectApiResponse({'took': 258, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 2595, 'relation': 'eq'}, 'max_score': 84.220634, 'hits': [{'_index': 'course', '_id': 'Z_FxaZABiBAFJm2dtVQI', '_score': 84.220634, '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}}, {'_index': 'course', '_id': 'G_F8aZABiBAFJm2dJFg7', '_score': 84.220634, '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoi

In [33]:
search_query={
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

In [34]:
response= es_client.search(index=index_name, body=search_query)

In [35]:
results_docs=[]
for hit in response["hits"]["hits"]:
    results_docs.append(hit["_source"])

In [44]:
results_docs

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': 'Launch the container im

In [48]:
context = """
Q: {question}
A: {text}
""".strip()

In [49]:
context =""

for doc in results_docs:
    context=context+ f"Q:{doc['question']}\nA: {doc['text']}\n\n"

In [50]:
print(context)

Q:How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q:How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q:How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker 

In [60]:
query="How do I execute a command in a running docker container?"

In [51]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [61]:
prompt=prompt_template.format(question=query, context=context).strip()

In [63]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do I execute a command in a running docker container?

CONTEXT:
Q:How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q:How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q:How do I debug a docker container?
A

In [66]:
response=client.chat.completions.create(
    model="gpt-4o",
    messages=[{"role":"user","content": prompt}]
)

In [67]:
response.choices[0].message.content

"To execute a command in a running Docker container, you can follow these steps:\n\n1. First, identify the container ID of the running Docker container. You can do this by using the `docker ps` command, which lists all running containers.\n\n    ```\n    docker ps\n    ```\n\n2. Once you have the container ID, use the `docker exec` command to execute a command within the running container. Here's the basic syntax for entering the container with a bash shell:\n\n    ```\n    docker exec -it <container-id> bash\n    ```\n    \nBy replacing `<container-id>` with the actual ID of your running container, you can execute the bash command or any other command within that container.\n\nFor example:\n    \n```sh\ndocker exec -it abc123 bash\n```"

In [68]:
len(response.choices[0].message.content)

725

In [72]:
import tiktoken

In [73]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [76]:
num_tokens=encoding.encode(response.choices[0].message.content)

In [77]:
len(num_tokens)

157